# Ingest Circuits Data

In [0]:
dbutils.widgets.text("env", "dev", "Environment")

env = dbutils.widgets.get("env")

In [0]:
%run ../config $env=$env

In [0]:
csv_file_path = raw_data_folder_path + "circuits.csv"

df_circuits = (
    spark.read
    .format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .load(csv_file_path)
)

df_circuits.display() if env == "dev" else None

In [0]:
from pyspark.sql.functions import current_timestamp

df_circuits_formatted = (
    df_circuits
    .withColumnRenamed("name", "circuit_name")
    .withColumnRenamed("circuitId", "circuit_id")
    .withColumnRenamed("circuitRef", "circuit_ref")
    .withColumnRenamed("lat", "latitude")
    .withColumnRenamed("lng", "longitude")
    .withColumnRenamed("alt", "altitude")
    .withColumn("ingestion_timestamp", current_timestamp())
    .drop("url")
)

df_circuits_formatted.display() if env == "dev" else None

In [0]:
df_circuits_formatted.write.format("delta").saveAsTable("bronze_tbl_circuits", mode="overwrite")